In [172]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('../data/linkfree_combined.csv')

In [173]:
df.head()

,type,linkfree_combined
0,INFJ,enfp intj moment sportscenter top ten play pra...
1,ENTP,im finding lack post alarming|||sex boring pos...
2,INTP,good one|||course say know thats blessing curs...
3,INTJ,dear intp enjoyed conversation day esoteric ga...
4,ENTJ,youre fired|||thats another silly misconceptio...


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   type               8675 non-null   object
 1   linkfree_combined  8674 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


In [175]:
df = df.dropna()

In [176]:
X = df.linkfree_combined.tolist()

In [177]:
df['tokenized'] = df.linkfree_combined.apply(tokenize_for_w2v)

In [178]:
def tokenize_for_w2v(linkfree_post):
    linkfree_post.replace('|||', ' ')
    return linkfree_post.split()

In [179]:
df.head()

,type,linkfree_combined,tokenized
0,INFJ,enfp intj moment sportscenter top ten play pra...,"[enfp, intj, moment, sportscenter, top, ten, p..."
1,ENTP,im finding lack post alarming|||sex boring pos...,"[im, finding, lack, post, alarming|||sex, bori..."
2,INTP,good one|||course say know thats blessing curs...,"[good, one|||course, say, know, thats, blessin..."
3,INTJ,dear intp enjoyed conversation day esoteric ga...,"[dear, intp, enjoyed, conversation, day, esote..."
4,ENTJ,youre fired|||thats another silly misconceptio...,"[youre, fired|||thats, another, silly, misconc..."


In [180]:
X = df.tokenized.tolist()

In [181]:
le = LabelEncoder()

In [182]:
y = df['type']
y = le.fit_transform(y)

In [183]:
#split data into train and test
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_split,y,test_size = 0.3, random_state = 1)

# Embedding with Word2Vec

In [186]:
type(X_train)

list

In [187]:
X_train[:2]

[['used',
  'test',
  'number',
  'post',
  'document',
  'result',
  'varied',
  'intj',
  'intp',
  'also',
  'used',
  'test',
  'passage',
  'jung',
  'psychological',
  'type',
  'got',
  'result',
  'intj|||format',
  'original',
  'article',
  'would',
  'conclude',
  'individual',
  'wrote',
  'underdeveloped',
  'asthetic',
  'sense',
  'real',
  'concern',
  'making',
  'world',
  'better|||absolutely',
  'chart',
  'table',
  'sort',
  'diagram',
  'help',
  'give',
  'overall',
  'view',
  'system',
  'galnce',
  'way',
  'simplifying',
  'data|||dont',
  'think',
  'need',
  'say',
  'anything',
  'tongue|||im',
  'usually',
  'good',
  'disappearing',
  'trick',
  'corner',
  'book|||ok',
  'nobody',
  'want',
  'typed',
  'jesus',
  'everybody',
  'want',
  'typed',
  'hitler|||clippered',
  'hair',
  'really',
  'short',
  'left',
  'mohawk',
  'made',
  'blond',
  'scary',
  'look|||ive',
  'always',
  'wanted',
  'set',
  'animal',
  'eye',
  'lens',
  'go',
  'clubbi

In [188]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

In [189]:
len(word2vec.wv.vocab.keys()), word2vec.wv.vocab.keys()

(24183,
 dict_keys(['used', 'test', 'number', 'post', 'document', 'result', 'varied', 'intj', 'intp', 'also', 'passage', 'jung', 'psychological', 'type', 'got', 'original', 'article', 'would', 'conclude', 'individual', 'wrote', 'underdeveloped', 'sense', 'real', 'concern', 'making', 'world', 'chart', 'table', 'sort', 'diagram', 'help', 'give', 'overall', 'view', 'system', 'way', 'simplifying', 'think', 'need', 'say', 'anything', 'tongue|||im', 'usually', 'good', 'disappearing', 'trick', 'corner', 'nobody', 'want', 'typed', 'jesus', 'everybody', 'hair', 'really', 'short', 'left', 'mohawk', 'made', 'blond', 'scary', 'look|||ive', 'always', 'wanted', 'set', 'animal', 'eye', 'lens', 'go', 'ive', 'found', 'helpful', 'learning', 'complex', 'mathematical', 'formula', 'work', 'understand', 'application', 'trouble', 'known', 'couple', 'people', 'thought', 'discussing', 'exploring', 'approach', 'decision', 'becomes', 'clear', 'indeed', 'infj', 'screamed', 'bully', 'wet', 'self', 'started', 'atti

### Convert X on Model Ready Data

In [190]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

In [191]:
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed


X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

## Padding

In [192]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post')